In [91]:
import os  # type: ignore
import sys  # type: ignore
import pandas as pd # type: ignore
import numpy as np  # type: ignore
import math
from sklearn.preprocessing import OneHotEncoder # type: ignore


# ============================================================
# ✅ Add the project root to sys.path (not /core)
# ============================================================
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

Project root added to sys.path: /Users/sadiqqais/Masterschool/Projekt/Mastery_projekt/Travel_tide


In [92]:
# Verify that core is importable
try:
    import core
    print("✅ core module found at:", core.__file__)
except ModuleNotFoundError:
    print("❌ core module not found. Check your sys.path!")

✅ core module found at: /Users/sadiqqais/Masterschool/Projekt/Mastery_projekt/Travel_tide/core/__init__.py


In [93]:
# Pfad für Feature-Metriken
feature_metrics_path = os.path.join(project_root, 'data', 'processed', 'feature_metrics')
#dashboard_path = os.path.join(eda_figures_path, 'dashboard')
#eda_data_path = os.path.join(project_root, 'reports', 'eda', 'results')
os.makedirs(feature_metrics_path, exist_ok=True)
#os.makedirs(dashboard_path, exist_ok=True)
#os.makedirs(eda_data_path, exist_ok=True)

In [94]:
# --- Helper methods ---
def haversine(lat1, lon1, lat2, lon2):
        """Calculate distance between two coordinates using Haversine formula."""
        R = 6371  # Earth radius in kilometers
        try:
            lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            return R * c
        except (ValueError, TypeError):
            return np.nan

def group_trip(row):
  if row['flight_booked'] == True and row['return_flight_booked'] == True and row['hotel_booked'] == True and row['seats'] > 2 and row['rooms'] > 1:  # noqa: E712
    return 1
  #Bus/Bahn/Auto Reise
  elif row['flight_booked'] == False and row['hotel_booked'] == True and row['rooms'] > 1 :  # noqa: E712
    return 1
  else:
    return 0

def pair_trip(row):
  if row['flight_booked'] == True and row['return_flight_booked'] == True and row['hotel_booked'] == True and not pd.isna(row['seats']) and not pd.isna(row['rooms']) and row['seats'] == 2 and row['rooms']==1:  # noqa: E712
    return 1
  else:
    return 0

def business_week_trip(row):
  birth_date = pd.to_datetime(row['birthdate'], format='mixed')
  age = (pd.Timestamp.today() - birth_date).days / 365
  departure = row['departure_time']
  return_ = row['return_time']
  #check for flights during the week
  ## Weekday: Monday = 0, Sunday = 6
  # Business days are 0 to 4 (Monday to Friday)
  if row['flight_booked'] == True and row['return_flight_booked'] == True and row['hotel_booked'] == True and row['seats'] == 1 and row['nights'] >= 1 and row['nights'] < 6 and age >= 25 and age <= 60 and (departure.weekday() <= 4) and (return_.weekday() <= 4):  # noqa: E712
    return 1
  else:
    return 0

def weekend_trip(row):
  departure = row['departure_time']
  return_ = row['return_time']
  # Friday = 4, Sunday = 6
  if row['flight_booked'] == True and row['return_flight_booked'] == True and row['hotel_booked'] == True and not pd.isna(row['nights']) and row['nights'] <= 2 and (departure.weekday() >= 4) and (return_.weekday() <= 6):  # noqa: E712
    return 1
  else:
    return 0

def season_trip(row):
  if row['departure_time'].month in [12, 1, 2]:
    return "winter"
  if row['departure_time'].month in [6, 7, 8]:
    return "summer"
  if row['departure_time'].month in [9, 10, 11]:
    return "fall"
  else:
    return "spring"

In [95]:

from core.load_data import load_table  # type: ignore

In [96]:
users = load_table(data_type='raw', table_name='users')
df_sessions = load_table(data_type='processed', table_name='sessions_cleaned')
df_nc_sessions = load_table(data_type='processed', table_name='sessions_not_canceled_trips')

:aktenordner: Lade Tabelle 'users' aus CSV: /Users/sadiqqais/Masterschool/Projekt/Mastery_projekt/Travel_tide/core/../data/raw/users.csv
:weißes_häkchen: CSV geladen. Zeilen: 1020926
:aktenordner: Lade Tabelle 'sessions_cleaned' aus CSV: /Users/sadiqqais/Masterschool/Projekt/Mastery_projekt/Travel_tide/core/../data/processed/sessions_cleaned.csv
:weißes_häkchen: CSV geladen. Zeilen: 49211
:aktenordner: Lade Tabelle 'sessions_not_canceled_trips' aus CSV: /Users/sadiqqais/Masterschool/Projekt/Mastery_projekt/Travel_tide/core/../data/processed/sessions_not_canceled_trips.csv
:weißes_häkchen: CSV geladen. Zeilen: 16099


In [97]:
df_sessions.columns

Index(['session_id', 'user_id', 'trip_id', 'session_start', 'session_end',
       'page_clicks', 'flight_discount', 'flight_discount_amount',
       'hotel_discount', 'hotel_discount_amount', 'flight_booked',
       'hotel_booked', 'cancellation', 'birthdate', 'gender', 'married',
       'has_children', 'home_country', 'home_city', 'home_airport',
       'home_airport_lat', 'home_airport_lon', 'sign_up_date',
       'origin_airport', 'destination', 'destination_airport', 'seats',
       'return_flight_booked', 'departure_time', 'return_time', 'checked_bags',
       'trip_airline', 'destination_airport_lat', 'destination_airport_lon',
       'base_fare_usd', 'hotel_name', 'nights', 'rooms', 'check_in_time',
       'check_out_time', 'hotel_price_per_room_night_usd', 'session_duration'],
      dtype='object')

In [98]:

#matches best possible value types
df_nc_sessions = df_nc_sessions.convert_dtypes()

#manuell für TimeObjects
#df_session
df_sessions['session_start'] = pd.to_datetime(df_sessions['session_start'])
df_sessions['session_end'] = pd.to_datetime(df_sessions['session_end'], format = 'mixed')
df_sessions['departure_time'] = pd.to_datetime(df_sessions['departure_time'], format = 'mixed')
#df_sessions['session_duration'] = (df_sessions['session_end'] - df_sessions['session_start']).dt.total_seconds()

#df_not_canceled_trips
df_nc_sessions['session_end'] = pd.to_datetime(df_nc_sessions['session_end'], format = 'mixed')
df_nc_sessions['departure_time'] = pd.to_datetime(df_nc_sessions['departure_time'])
df_nc_sessions['return_time'] = pd.to_datetime(df_nc_sessions['return_time'], format = 'mixed')
df_nc_sessions['check_in_time'] = pd.to_datetime(df_nc_sessions['check_in_time'], format = 'mixed')

In [99]:

users['birthdate'] = pd.to_datetime(users['birthdate'], format='mixed')
df_sessions['session_duration'] = (df_sessions['session_end'] - df_sessions['session_start']).dt.total_seconds()

In [100]:
# CTE 1: canceled_trips
df_sessions['empty_session'] = df_sessions['trip_id'].isna().astype(int)

nc_trip_ids = df_nc_sessions['trip_id'].unique()
trip_ids = df_sessions.dropna(subset=['trip_id'])['trip_id'].unique()

canceled_trip_ids = trip_ids[~pd.Series(trip_ids).isin(nc_trip_ids)]

print(f'{len(nc_trip_ids)} not canceled trips; {len(canceled_trip_ids)} canceled trips')

df_sessions['canceled_trip'] = df_sessions['trip_id'].isin(canceled_trip_ids).astype(int)


16099 not canceled trips; 0 canceled trips


In [101]:
# CTE 3: user_base_session
# Aggregieren der Benutzersitzungsmetriken
user_base = df_sessions.groupby('user_id').agg(
    num_clicks= ('page_clicks', 'sum'),
    avg_session_clicks= ('page_clicks', 'mean'),
    num_empty_sessions = ('empty_session', 'sum'),
    num_sessions=('session_id', 'nunique'),
    num_canceled_trips= ('canceled_trip', 'sum'),
    avg_session_duration=('session_duration', 'mean')
).reset_index()

user_base.head()

,user_id,num_clicks,avg_session_clicks,num_empty_sessions,num_sessions,num_canceled_trips,avg_session_duration
0,23557,82,10.250,6,8,0,76.625000
1,94883,73,9.125,6,8,0,67.750000
2,101486,131,16.375,6,8,0,122.250000
3,101961,126,15.750,3,8,0,117.750000
4,106907,240,30.000,6,8,0,758.915066


In [102]:
# Fill missing boolean values with False before the operations
df_nc_sessions['flight_booked'] = df_nc_sessions['flight_booked'].fillna(False)
df_nc_sessions['return_flight_booked'] = df_nc_sessions['return_flight_booked'].fillna(False)
df_nc_sessions['hotel_discount'] = df_nc_sessions['hotel_discount'].fillna(False)
df_nc_sessions['flight_discount'] = df_nc_sessions['flight_discount'].fillna(False)
df_nc_sessions['hotel_booked'] = df_nc_sessions['hotel_booked'].fillna(False)




In [103]:
# df_nc_sessions['num_flights'] = np.where(
#     (df_nc_sessions['flight_booked'] == True) & (df_nc_sessions['return_flight_booked'] == True), 2, # noqa: E712
#     np.where(
#         (df_nc_sessions['flight_booked'] == True) & (df_nc_sessions['return_flight_booked'] == False), 1, 0 # noqa: E712
#     )
# )
df_nc_sessions['num_flights'] = np.where(
    (df_nc_sessions['flight_booked'] == True) & (df_nc_sessions['return_flight_booked'] == True), 2, 
    np.where(
        (df_nc_sessions['flight_booked'] == True) & (df_nc_sessions['return_flight_booked'] == False), 1, 0 
    )
)

df_nc_sessions["discount_trip"] = np.where(
    (df_nc_sessions['hotel_discount'] == True) | (df_nc_sessions['flight_discount'] == True),1,0  # noqa: E712
)

df_nc_sessions['num_hotels'] = np.where(
    (df_nc_sessions['hotel_booked'] == True), 1, 0  # noqa: E712
)

df_nc_sessions['money_spent_per_flight'] = np.where(
    df_nc_sessions['flight_discount'],  # condition: discount applies
    df_nc_sessions['base_fare_usd'] * (1 - df_nc_sessions['flight_discount_amount']),  # discounted price
    df_nc_sessions['base_fare_usd']  # no discount
)


df_nc_sessions['money_spent_per_seat'] = df_nc_sessions['money_spent_per_flight'] / df_nc_sessions['seats']

base_hotel_cost = (
    df_nc_sessions['hotel_price_per_room_night_usd'] *
    df_nc_sessions['nights'] *
    df_nc_sessions['rooms']
)

df_nc_sessions['money_spent_hotel'] = np.where(
    df_nc_sessions['hotel_discount'],  # condition: discount applies
    base_hotel_cost * (1 - df_nc_sessions['hotel_discount_amount']),  # discounted
    base_hotel_cost  # full price
)

df_nc_sessions['time_after_booking'] = (df_nc_sessions['departure_time'] - df_nc_sessions['session_end']).dt.days
df_nc_sessions['distance_km'] = df_nc_sessions.apply(lambda row: haversine(
    row['home_airport_lat'], 
    row['home_airport_lon'], 
    row['destination_airport_lat'], 
    row['destination_airport_lon']), 
    axis=1)

In [104]:
user_base_2 = df_nc_sessions.groupby('user_id').agg(
    #unique counts
    num_trips= ('trip_id', 'nunique'),
    num_destinations=('destination', 'nunique'),
    #sums
    num_flights= ('num_flights', 'sum'),
    num_hotels= ('num_hotels', 'sum'),
    #num_group_trips= ('group_trip', 'sum'),
    #num_pair_trips= ('pair_trip', 'sum'),
    #num_business_week_trips= ('business_week_trip', 'sum'),
    #num_weekend_trips= ('weekend_trip', 'sum'),
    num_discount_trips= ('discount_trip', 'sum'),
    #money_spent_hotel= ('money_spent_hotel', 'sum'),
    #money_spent_flight= ('money_spent_per_flight', 'sum'),
    #num_summer_trips = ('season_summer', 'sum'),
    #num_winter_trips = ('season_winter', 'sum'),
    #num_fall_trips = ('season_fall', 'sum'),
    #num_spring_trips = ('season_spring', 'sum'),
    #means
    avg_money_spent_flight= ('money_spent_per_flight', 'mean'),
    avg_time_after_booking= ('time_after_booking', 'mean'),
    avg_money_spent_per_seat= ('money_spent_per_seat', 'mean'),
    avg_money_spent_hotel= ('money_spent_hotel', 'mean'),
    avg_km_flown=('distance_km', 'mean'),
    avg_bags=('checked_bags','mean')
    ).reset_index()

print(user_base_2.shape)
print(user_base_2.isnull().sum())

user_base_2.head()

(5542, 12)
user_id                       0
num_trips                     0
num_destinations              0
num_flights                   0
num_hotels                    0
num_discount_trips            0
avg_money_spent_flight      367
avg_time_after_booking      362
avg_money_spent_per_seat    367
avg_money_spent_hotel       146
avg_km_flown                362
avg_bags                    362
dtype: int64


,user_id,num_trips,num_destinations,num_flights,num_hotels,num_discount_trips,avg_money_spent_flight,avg_time_after_booking,avg_money_spent_per_seat,avg_money_spent_hotel,avg_km_flown,avg_bags
0,23557,2,0,0,2,1,NaN,NaN,<NA>,1835.25,NaN,<NA>
1,94883,2,2,4,2,0,432.0450,6.5,276.2525,115.00,1453.666128,0.5
2,101486,2,1,2,2,0,189.9100,9.0,189.91,1099.50,965.340568,0.0
3,101961,5,5,10,5,1,247.5386,5.6,247.5386,485.80,1321.684183,0.4
4,106907,1,1,2,1,0,13902.0600,224.0,2317.01,4257.00,13402.323077,5.0


## **user_base_2.describe().T**


In [105]:
user_base_2.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,5542.0,544487.424035,64711.276003,23557.0,516966.75,539788.0,573140.75,811077.0
num_trips,5542.0,2.904908,1.364653,1.0,2.0,3.0,4.0,8.0
num_destinations,5542.0,2.34446,1.324966,0.0,1.0,2.0,3.0,8.0
num_flights,5542.0,4.844641,2.794574,0.0,2.0,4.0,6.0,16.0
num_hotels,5542.0,2.583724,1.342774,0.0,2.0,2.0,3.0,8.0
num_discount_trips,5542.0,0.774811,0.820791,0.0,0.0,1.0,1.0,6.0
avg_money_spent_flight,5175.0,515.137864,677.284721,5.35,275.111667,385.04,548.325917,14280.38
avg_time_after_booking,5180.0,19.663191,46.205595,2.0,6.0,7.0,8.5,365.0
avg_money_spent_per_seat,5175.0,396.829987,279.09518,5.35,249.81625,347.33,467.117292,3026.45
avg_money_spent_hotel,5396.0,834.629319,938.634504,27.9,329.716667,558.125,971.75,14057.0


In [106]:
users['birthdate'] = pd.to_datetime(users['birthdate'], format='mixed')
users['age'] = (pd.Timestamp.today() - users['birthdate']).dt.days / 365

In [107]:
df_user_base = pd.merge(user_base, user_base_2, on='user_id', how='left')
df_user_base = pd.merge(df_user_base, users, on='user_id', how='left')
print(df_user_base.shape)
print(df_user_base.isnull().sum())
print(df_user_base.columns)
df_user_base.head()

(5998, 29)
user_id                       0
num_clicks                    0
avg_session_clicks            0
num_empty_sessions            0
num_sessions                  0
num_canceled_trips            0
avg_session_duration          0
num_trips                   456
num_destinations            456
num_flights                 456
num_hotels                  456
num_discount_trips          456
avg_money_spent_flight      823
avg_time_after_booking      818
avg_money_spent_per_seat    823
avg_money_spent_hotel       602
avg_km_flown                818
avg_bags                    818
birthdate                     0
gender                        0
married                       0
has_children                  0
home_country                  0
home_city                     0
home_airport                  0
home_airport_lat              0
home_airport_lon              0
sign_up_date                  0
age                           0
dtype: int64
Index(['user_id', 'num_clicks', 'avg_session_cli

,user_id,num_clicks,avg_session_clicks,num_empty_sessions,num_sessions,num_canceled_trips,avg_session_duration,num_trips,num_destinations,num_flights,...,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,age
0,23557,82,10.250,6,8,0,76.625000,2.0,0.0,0.0,...,F,True,False,usa,new york,LGA,40.777,-73.872,2021-07-22,67.010959
1,94883,73,9.125,6,8,0,67.750000,2.0,2.0,4.0,...,F,True,False,usa,kansas city,MCI,39.297,-94.714,2022-02-07,53.731507
2,101486,131,16.375,6,8,0,122.250000,2.0,1.0,2.0,...,F,True,True,usa,tacoma,TCM,47.138,-122.476,2022-02-17,53.002740
3,101961,126,15.750,3,8,0,117.750000,5.0,5.0,10.0,...,F,True,False,usa,boston,BOS,42.364,-71.005,2022-02-17,45.227397
4,106907,240,30.000,6,8,0,758.915066,1.0,1.0,2.0,...,F,True,True,usa,miami,TNT,25.862,-80.897,2022-02-24,47.054795


In [108]:
#columns we dont need for our final frame

columns = ['birthdate', 'home_airport', 'home_airport_lat',
       'home_airport_lon', 'sign_up_date']

df_user_base.drop(columns, axis=1, inplace=True)

save_path = os.path.join(feature_metrics_path, "user_base.csv")
df_user_base.to_csv(save_path, index=False)



In [109]:
df_user_base.describe()

,user_id,num_clicks,avg_session_clicks,num_empty_sessions,num_sessions,num_canceled_trips,avg_session_duration,num_trips,num_destinations,num_flights,num_hotels,num_discount_trips,avg_money_spent_flight,avg_time_after_booking,avg_money_spent_per_seat,avg_money_spent_hotel,avg_km_flown,avg_bags,age
count,5998.000000,5998.000000,5998.000000,5998.000000,5998.000000,5998.0,5998.000000,5542.000000,5542.000000,5542.000000,5542.000000,5542.000000,5175.000000,5180.000000,5175.0,5396.000000,5180.000000,5180.0,5998.000000
mean,545440.695565,144.308436,17.586393,5.419973,8.204568,0.0,187.060316,2.904908,2.344460,4.844641,2.583724,0.774811,515.137864,19.663191,396.829987,834.629319,2327.958231,0.596612,43.734047
std,64924.749760,72.330165,8.735165,1.611049,0.460911,0.0,241.058619,1.364653,1.324966,2.794574,1.342774,0.820791,677.284721,46.205595,279.09518,938.634504,1624.664922,0.500549,12.051886
min,23557.000000,33.000000,4.125000,0.000000,8.000000,0.0,31.500000,1.000000,0.000000,0.000000,0.000000,0.000000,5.350000,2.000000,5.35,27.900000,27.440552,0.0,18.923288
25%,517152.000000,102.000000,12.500000,4.000000,8.000000,0.0,93.000000,2.000000,1.000000,2.000000,2.000000,0.000000,275.111667,6.000000,249.81625,329.716667,1485.709109,0.25,36.658219
50%,540542.000000,127.000000,15.625000,5.000000,8.000000,0.0,116.375000,3.000000,2.000000,4.000000,2.000000,1.000000,385.040000,7.000000,347.33,558.125000,2033.759929,0.5,44.127397
75%,574146.000000,161.000000,19.625000,7.000000,8.000000,0.0,147.375000,4.000000,3.000000,6.000000,3.000000,1.000000,548.325917,8.500000,467.117292,971.750000,2735.357181,1.0,51.199315
max,844489.000000,873.000000,109.125000,12.000000,12.000000,0.0,2137.625000,8.000000,8.000000,16.000000,8.000000,6.000000,14280.380000,365.000000,3026.45,14057.000000,15935.659875,5.0,90.608219


In [110]:
df_user_base.columns

Index(['user_id', 'num_clicks', 'avg_session_clicks', 'num_empty_sessions',
       'num_sessions', 'num_canceled_trips', 'avg_session_duration',
       'num_trips', 'num_destinations', 'num_flights', 'num_hotels',
       'num_discount_trips', 'avg_money_spent_flight',
       'avg_time_after_booking', 'avg_money_spent_per_seat',
       'avg_money_spent_hotel', 'avg_km_flown', 'avg_bags', 'gender',
       'married', 'has_children', 'home_country', 'home_city', 'age'],
      dtype='object')